In [1]:
from mido import Message, MidiFile, MidiTrack
import os

In [2]:
import pickle
with open('jsb-chorales-16th.pkl', 'rb') as p:
    data = pickle.load(p, encoding="latin1")

In [3]:
def notes_to_midi(note_array, outpath):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    TICKS_PER_BEAT = 480
    step_time = TICKS_PER_BEAT // 4  # 1 beat per step
    velocity = 64

    previous_notes = set()
    curr_time = 0
    for chord in note_array:
        current_notes = set(chord)
        current_notes = {int(n) for n in current_notes}

        # Notes to turn off (not in current chord)
        for note in previous_notes - current_notes:
            track.append(Message('note_off', note=note, velocity=velocity, time=curr_time))

        # Notes to turn on (new in current chord)
        for note in current_notes - previous_notes:
            track.append(Message('note_on', note=note, velocity=velocity, time=curr_time))

        previous_notes = current_notes
        curr_time += step_time

    # Turn off all remaining notes at the end
    for note in previous_notes:
        track.append(Message('note_off', note=note, velocity=velocity, time=curr_time))

    mid.save(outpath)

notes_to_midi(data['train'][0], "output.mid")
print(data.keys())

dict_keys(['test', 'train', 'valid'])


In [7]:
try:
    os.mkdir("train_data")
    os.mkdir("valid_data")
    os.mkdir("test_data")
    print(f"Folders created successfully.")
except FileExistsError:
    print(f"Folders already exists.")
except Exception as e:
    print(f"An error occurred: {e}")



Folders created successfully.


In [9]:
for i in range(len(data['train'])):
    notes_to_midi(data['train'][i], f"train_data/train_{i:02d}.mid")
for i in range(len(data['valid'])):
    notes_to_midi(data['valid'][i], f"valid_data/valid_{i:02d}.mid")
for i in range(len(data['test'])):
    notes_to_midi(data['test'][i], f"test_data/test_{i:02d}.mid")